In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_naver_finance_board(codes, max_page, year, month, day):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'
    }

    time_pass_flag = 0
    result = []

    target_date = datetime(year, month, day).date()

    for page_num in range(1, max_page + 1):
        url = f"https://finance.naver.com/item/board.naver?code={codes}&page={page_num}"
        try:
            response = requests.get(url, headers=headers)
            html = response.content
            soup = BeautifulSoup(html.decode('euc-kr', 'replace'), 'html.parser')
            table = soup.find('table', {'class': 'type2'})
            rows = table.select('tbody > tr')
            for row in rows[2:]:
                cols = row.select('td')
                if len(cols) >= 6:
                    date = cols[0].select_one('span').get_text(strip=True)
                    datetime_object = datetime.strptime(date, "%Y.%m.%d %H:%M")
                    if datetime_object.date() < target_date:
                        time_pass_flag = 1
                        break
                    title = cols[1].select_one('a').get('title').strip()
                    views = cols[3].select_one('span').get_text(strip=True)
                    upvote = cols[4].select_one('strong').get_text(strip=True)
                    downvote = cols[5].select_one('strong').get_text(strip=True)
                    result.append([date, title, views, upvote, downvote])

            if time_pass_flag:
                break
        except requests.exceptions.RequestException as e:
            print(f"Error occurred while fetching page {page_num}: {e}")
            break

    df = pd.DataFrame(result, columns=['날짜', '제목', '조회', '공감', '비공감'])
    
    # combine titles by same date
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y.%m.%d %H:%M').dt.date
    df['제목'] = df.groupby('날짜')['제목'].transform(lambda x: ', '.join(x))
    df.drop_duplicates(subset='날짜', keep='first', inplace=True)

    df.drop(columns = ['조회', '공감', '비공감'], inplace=True)

    return df

In [3]:
get_naver_finance_board("066970", 30, 2023, 2, 1)

,날짜,제목
0,2023-03-23,"머스크 형 덕에, 엘엔에프 어디까지 가는거야, ■■폭스바겐 “25년부터 5년간 소형..."
4,2023-03-22,"왜 이렇게 못가는겨 ㅡ, 한마디로 요약하자면, 참 희한한게, 늦은시간까지 고민중이시..."


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

# Define the labels for the classifier
labels = ['Negative', 'Neutral', 'Positive']

# Define a sample sentence to classify
sentence = "한국과 일본 증시가 휴장한 가운데 2008년 마지막 거래일인 31일 오전 아시아 주."

# Load the sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

# Use the pipeline to classify the sentiment of the input sentence
result = sentiment_analysis(sentence)

max_score_label = max(result[0], key=lambda x: x['score'])['label']

print(result[0], max_score_label)



[{'label': 'negative', 'score': 0.0019540241919457912}, {'label': 'neutral', 'score': 0.9978947043418884}, {'label': 'positive', 'score': 0.0001512996241217479}] neutral


In [29]:
from transformers import pipeline

# Load the sentiment analysis pipeline for Korean
sentiment_analysis = pipeline("sentiment-analysis", model="snunlp/KR-FinBert-SC")

# Define a sample sentence to classify
sentence = "이 영화 너무 별로였어."

# Use the pipeline to classify the sentiment of the input sentence
result = sentiment_analysis(sentence)

# Print the predicted sentiment and its score
print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']}")


Sentiment: neutral, Score: 0.9369769096374512


In [26]:
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline


# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

# Define the labels for the classifier
labels = ['Negative', 'Neutral', 'Positive']

# Load the sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

def get_sentiment_analysis(text):
    # Classify the sentiment of the input sentence
    if len(text) > 512:
        text = text[:512] # restrict model input 512
    result = sentiment_analysis(text)

    max_score_label = max(result[0], key=lambda x: x['score'])['label']

    score_dict = {'negative': -1, 'neutral': 0, 'positive': 1}
    score = sum([score_dict[r['label']] * r['score'] for r in result[0]])

    # Return highest confidence score label
    return score


def get_naver_finance_board(codes, max_page, year, month, day):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'
    }

    time_pass_flag = 0
    result = []

    target_date = datetime(year, month, day).date()

    for page_num in range(1, max_page + 1):
        url = f"https://finance.naver.com/item/board.naver?code={codes}&page={page_num}"
        try:
            response = requests.get(url, headers=headers)
            html = response.content
            soup = BeautifulSoup(html.decode('euc-kr', 'replace'), 'html.parser')
            table = soup.find('table', {'class': 'type2'})
            rows = table.select('tbody > tr')
            for row in rows[2:]:
                cols = row.select('td')
                if len(cols) >= 6:
                    date = cols[0].select_one('span').get_text(strip=True)
                    datetime_object = datetime.strptime(date, "%Y.%m.%d %H:%M")
                    if datetime_object.date() < target_date:
                        time_pass_flag = 1
                        break
                    title = cols[1].select_one('a').get('title').strip()
                    views = cols[3].select_one('span').get_text(strip=True)
                    upvote = cols[4].select_one('strong').get_text(strip=True)
                    downvote = cols[5].select_one('strong').get_text(strip=True)
                    result.append([date, title, views, upvote, downvote])

            if time_pass_flag:
                break
        except requests.exceptions.RequestException as e:
            print(f"Error occurred while fetching page {page_num}: {e}")
            break

    df = pd.DataFrame(result, columns=['날짜', '제목', '조회', '공감', '비공감'])
    # combine titles by same date
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y.%m.%d %H:%M').dt.date
    # df.drop_duplicates(subset='날짜', keep='first', inplace=True)

    df.drop(columns = ['조회', '공감', '비공감'], inplace=True)

    df['감성분석'] = df['제목'].apply(get_sentiment_analysis)

    # df['감성분석'] = df.groupby('날짜')['감성분석'].sum().reset_index()

    print(df)
    return df

In [27]:
get_naver_finance_board("066970", 30, 2023, 2, 1)

                   날짜                                 제목   조회 공감 비공감
0    2023.03.23 00:37                  ◆◆배터리아저씨 박순헉이사 탑픽   65  0   2
1    2023.03.23 00:23                           머스크 형 덕에   48  0   0
2    2023.03.23 00:21                     엘엔에프 어디까지 가는거야   56  0   0
3    2023.03.23 00:14  ■■폭스바겐 “25년부터 5년간 소형전기차 300만대 생산”   96  0   0
4    2023.03.23 00:10                                3공장   87  0   0
..                ...                                ...  ... ..  ..
595  2023.03.22 09:22        개미들이 안사니 더 올리고 있누 ㅋㅋㅋ 개미꼬시기   69  1   0
596  2023.03.22 09:22                                 오늘   31  0   0
597  2023.03.22 09:22          주가의 상승 하락 보다 시급한 건 경영 마인드   37  0   0
598  2023.03.22 09:21                                  찐   42  0   0
599  2023.03.22 09:20                               안되겠다  172  1   0

[600 rows x 5 columns]
             날짜                                 제목
0    2023-03-23                  ◆◆배터리아저씨 박순헉이사 탑픽
1    2023-03-23                           머스크 